In [28]:
import cvxpy as cp
import numpy as np
import pandas as pd
import yfinance as yf

print(cp.installed_solvers())

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'GUROBI', 'OSQP', 'SCIPY', 'SCS']


In [27]:
pip install cvxopt


In [22]:
def optimize_portfolio_with_risk(stocks, returns, cov_matrix, min_return, max_stocks):
    """
    Select a certain amount of stocks to minimize risk subject to a minimum expected return.

    Parameters:
    - stocks: List of stock symbols.
    - returns: Array of expected returns for each stock.
    - cov_matrix: Covariance matrix of stock returns.
    - min_return: Minimum expected return for the portfolio.
    - max_stocks: Maximum number of stocks to select.

    Returns:
    - A tuple of the optimized portfolio, its expected return, and its risk (standard deviation).
    """

    n = len(stocks) # Number of stocks
    # Decision variables
    x = cp.Variable(n, boolean=True) # Binary variables for stock selection
    w = cp.Variable(n) # Continuous variables for weights

    # Objective: Minimize portfolio risk (standard deviation)
    risk = cp.quad_form(w, cov_matrix)
    objective = cp.Minimize(risk)

    # Constraints
    constraints = [
        cp.sum(w) == 1, # Sum of weights is 1
        w >= 0, # No short selling
        w <= x, # Weight can be non-zero only if stock is selected
        cp.sum(x) == max_stocks, # Select exactly max_stocks stocks
        cp.sum(cp.multiply(w, returns)) >= min_return # Minimum expected return
    ]

    # Solve the problem
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.CVXOPT)

    # Check if a valid solution exists
    if prob.status == cp.OPTIMAL or prob.status == cp.OPTIMAL_INACCURATE:
        selected_stocks = [stocks[i] for i in range(n) if round(x.value[i]) == 1]
        portfolio_return = sum(w.value[i] * returns[i] for i in range(n))
        portfolio_risk = np.sqrt(risk.value)
        return selected_stocks, portfolio_return, portfolio_risk
    else:
        raise Exception("No optimal solution found.")

In [29]:
# REPLACE WITH REAL DATA
stocks = ['1', '2', '3', '4', '5']
expected_returns = np.array([0.1, 0.12, 0.15, 0.08, 0.2]) # This should be actual data
cov_matrix = np.random.rand(5, 5) # This should be an actual covariance matrix
cov_matrix = (cov_matrix + cov_matrix.T) / 2 # Ensure the matrix is symmetric
np.fill_diagonal(cov_matrix, cov_matrix.diagonal() * 10) # Increase variance on the diagonal for realism
min_return = 0.005
max_stocks = 3

selected_stocks, portfolio_return, portfolio_risk = optimize_portfolio_with_risk(stocks, expected_returns, cov_matrix, min_return, max_stocks)
print(f"Selected Stocks: {selected_stocks}")
print(f"Portfolio Expected Return: {portfolio_return}")
print(f"Portfolio Risk (Std. Dev.): {portfolio_risk}")

SolverError: The solver GUROBI is not installed.